### <center>ИССЛЕДОВАНИЕ ДАННЫХ HR-АГЕНТСТВА</center>

HR-агентство изучает тренды на рынке труда в IT. Компания хочет провести исследование на основе данных о зарплатах в сфере Data Science за 2020–2022 годы и получить некоторые выводы.
Исследуйте данные и сделайте выводы по полученным результатам. Подкрепите свои рассуждения и выводы визуализациями и с помощью статистического тестирования проверьте, являются ли выводы статистически значимыми.

В процессе своего анализа вы должны:

Выяснить, какие факторы влияют на зарплату специалиста Data Scientist.
А также ответить на ключевые вопросы HR-агентства:

1. Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
2. Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
3. Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
4. Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?
5. Если вы найдёте в данных интересные закономерности, также отметьте их в своём анализе.

Продемонстрируйте использование разных тестов для проверки статистической значимости сделанных выводов:

тесты для количественного признака:
для одной выборки;
для двух выборок;
для нескольких выборок;
тест для категориальных признаков.

In [174]:
!pip install pandas
!pip install numpy
!pip install plotly
!pip install --upgrade nbformat
!pip install -U ydata-profiling
!pip install ipywidgets
!pip install scipy



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [175]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ydata_profiling import ProfileReport
import scipy.stats as stats

## 1. Загрузка и обработка данных

### 1.1 Загрузка данных

In [176]:
#Загружаем данные
data = pd.read_csv('data/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ds_salaries.zip', index_col=0)

In [177]:
data.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M


In [178]:
# задаём уровень значимости
alpha = 0.05 
print(f"Уровень значимости alpha = {alpha}")

Уровень значимости alpha = 0.05


### 1.2 Описательный анализ данных

In [179]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    object
 2   employment_type     607 non-null    object
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    object
dtypes: int64(4), object(7)
memory usage: 56.9+ KB


work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

Всего имеем 607 записей. Пустых ячеек нет.

In [180]:
data.describe(include='all')

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
count,607.000000,607,607,607,6.070000e+02,607,607.000000,607,607.00000,607,607
unique,NaN,4,4,50,NaN,17,NaN,57,NaN,50,3
top,NaN,SE,FT,Data Scientist,NaN,USD,NaN,US,NaN,US,M
freq,NaN,280,588,143,NaN,398,NaN,332,NaN,355,326
mean,2021.405272,NaN,NaN,NaN,3.240001e+05,NaN,112297.869852,NaN,70.92257,NaN,NaN
std,0.692133,NaN,NaN,NaN,1.544357e+06,NaN,70957.259411,NaN,40.70913,NaN,NaN
min,2020.000000,NaN,NaN,NaN,4.000000e+03,NaN,2859.000000,NaN,0.00000,NaN,NaN
25%,2021.000000,NaN,NaN,NaN,7.000000e+04,NaN,62726.000000,NaN,50.00000,NaN,NaN
50%,2022.000000,NaN,NaN,NaN,1.150000e+05,NaN,101570.000000,NaN,100.00000,NaN,NaN
75%,2022.000000,NaN,NaN,NaN,1.650000e+05,NaN,150000.000000,NaN,100.00000,NaN,NaN


### 1.3 Очистка данных

Проверим данные на наличие полных дубликатов:

In [181]:
#Проверим данные на наличие полных дубликатов:
data.duplicated().sum()

42

Найдено 42 дубликата. Удаляем дубликаты.

In [182]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [183]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 565 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           565 non-null    int64 
 1   experience_level    565 non-null    object
 2   employment_type     565 non-null    object
 3   job_title           565 non-null    object
 4   salary              565 non-null    int64 
 5   salary_currency     565 non-null    object
 6   salary_in_usd       565 non-null    int64 
 7   employee_residence  565 non-null    object
 8   remote_ratio        565 non-null    int64 
 9   company_location    565 non-null    object
 10  company_size        565 non-null    object
dtypes: int64(4), object(7)
memory usage: 53.0+ KB


Имеем 565 записей без пропусков.

Рассмотрим распределение заработной платы для выявления выбросов.

In [184]:
fig = px.box(
    data,
    x = 'salary_in_usd',
    labels={'salary_in_usd': "зарплата (USD)", 'y': 'Распределение'},
    title='Распределение зароботной платы в долларах США',
    # marginal='box',
    )

fig.show()

In [185]:
fig = px.histogram(
    data,
    x = 'salary_in_usd',
    labels={'salary_in_usd': "зарплата (USD)", 'y': 'Распределение'},
    title='Распределение зароботной платы в долларах США',
    # marginal='box',
    )

fig.show()

Убираем выбросы по методу Тьюки. Распределение логнормально, так что пименима следующая функция:

In [186]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=True):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [187]:
_, data = outliers_iqr_mod(data, 'salary_in_usd')

In [188]:
fig = px.box(
    data,
    x = 'salary_in_usd',
    labels={'salary_in_usd': "зарплата (USD)", 'y': 'Распределение'},
    title='Распределение зароботной платы в долларах США',
    # marginal='box',
    )

fig.show()

In [189]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           543 non-null    int64 
 1   experience_level    543 non-null    object
 2   employment_type     543 non-null    object
 3   job_title           543 non-null    object
 4   salary              543 non-null    int64 
 5   salary_currency     543 non-null    object
 6   salary_in_usd       543 non-null    int64 
 7   employee_residence  543 non-null    object
 8   remote_ratio        543 non-null    int64 
 9   company_location    543 non-null    object
 10  company_size        543 non-null    object
dtypes: int64(4), object(7)
memory usage: 50.9+ KB


Итого осталось 543 записи.

In [190]:
data.describe(include='all')

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
count,543.000000,543,543,543,5.430000e+02,543,543.000000,543,543.000000,543,543
unique,NaN,4,4,49,NaN,17,NaN,54,NaN,46,3
top,NaN,SE,FT,Data Scientist,NaN,USD,NaN,US,NaN,US,M
freq,NaN,243,529,124,NaN,348,NaN,294,NaN,314,281
mean,2021.386740,NaN,NaN,NaN,3.440488e+05,NaN,113660.530387,NaN,70.441989,NaN,NaN
std,0.696306,NaN,NaN,NaN,1.631085e+06,NaN,67679.206211,NaN,40.706263,NaN,NaN
min,2020.000000,NaN,NaN,NaN,1.400000e+04,NaN,15966.000000,NaN,0.000000,NaN,NaN
25%,2021.000000,NaN,NaN,NaN,7.000000e+04,NaN,65000.000000,NaN,50.000000,NaN,NaN
50%,2022.000000,NaN,NaN,NaN,1.123000e+05,NaN,102100.000000,NaN,100.000000,NaN,NaN
75%,2022.000000,NaN,NaN,NaN,1.644980e+05,NaN,150000.000000,NaN,100.000000,NaN,NaN


Исходя из минимальных и максимальных значений по годам выплаты зарплаты и параметров удаленной работы, никаких лишних значений в этих колонках нет.

Проверяем уникальные значения для столбцов remote_ratio и work_year

In [191]:
display(data['remote_ratio'].unique())
display(data['work_year'].unique())

array([  0,  50, 100], dtype=int64)

array([2020, 2021, 2022], dtype=int64)

Переведем колонки experience_level, employment_type, company_size, employee_residence, company_location в тип Category. Salary_currency трогать не будем, так как колонка планируется под удаление на следующем этапе.

In [192]:
data['experience_level'] = data['experience_level'].astype('category')
data['employment_type'] = data['employment_type'].astype('category')
data['company_size'] = data['company_size'].astype('category')
data['employee_residence'] = data['employee_residence'].astype('category')
data['company_location'] = data['company_location'].astype('category')

Рассмотрим employee_residence.

In [193]:
data.groupby('employee_residence')['work_year'].count().sort_values(ascending=False)

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\3624560856.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



employee_residence
US    294
GB     43
CA     27
DE     24
IN     23
FR     18
ES     14
GR     12
JP      7
PT      5
NL      5
RU      4
PL      4
BR      4
IT      4
PK      3
AE      3
AU      3
AT      3
SG      2
BE      2
RO      2
HU      2
VN      2
SI      2
TR      2
DK      2
TN      1
NG      1
RS      1
PR      1
BG      1
BO      1
PH      1
NZ      1
CH      1
EE      1
MY      1
AR      1
CZ      1
CO      1
HK      1
HN      1
HR      1
IE      1
IQ      1
DZ      1
CN      1
JE      1
CL      1
LU      1
MD      1
MT      1
MX      1
Name: work_year, dtype: int64

Предлагаю сократить количество важных значений до 8, а остальные обозначить как 'other'.

In [194]:
country_list = ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']
data['employee_residence'] = data['employee_residence'].apply(lambda x: x if x in country_list else 'other')
data.groupby('employee_residence')['work_year'].count().sort_values(ascending=False)

employee_residence
US       294
other     88
GB        43
CA        27
DE        24
IN        23
FR        18
ES        14
GR        12
Name: work_year, dtype: int64

Рассмотрим company_location.

In [195]:
data.groupby('company_location')['work_year'].count().sort_values(ascending=False)

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\786288337.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



company_location
US    314
GB     46
CA     28
DE     27
IN     19
FR     15
ES     13
GR     10
JP      6
AT      4
PT      4
PL      4
NL      4
AE      3
DK      3
AU      3
MX      2
CZ      2
TR      2
CN      2
SI      2
RU      2
BR      2
BE      2
LU      2
IT      2
NZ      1
NG      1
DZ      1
PK      1
MY      1
SG      1
RO      1
MD      1
MT      1
EE      1
CH      1
IQ      1
CL      1
IL      1
AS      1
HU      1
HR      1
HN      1
CO      1
IE      1
Name: work_year, dtype: int64

Предлагаю поступить так же, как и с employee_residence

In [196]:
data['company_location'] = data['company_location'].apply(lambda x: x if x in country_list else 'other')
data.groupby('company_location')['work_year'].count().sort_values(ascending=False)

company_location
US       314
other     71
GB        46
CA        28
DE        27
IN        19
FR        15
ES        13
GR        10
Name: work_year, dtype: int64

In [197]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           543 non-null    int64   
 1   experience_level    543 non-null    category
 2   employment_type     543 non-null    category
 3   job_title           543 non-null    object  
 4   salary              543 non-null    int64   
 5   salary_currency     543 non-null    object  
 6   salary_in_usd       543 non-null    int64   
 7   employee_residence  543 non-null    object  
 8   remote_ratio        543 non-null    int64   
 9   company_location    543 non-null    object  
 10  company_size        543 non-null    category
dtypes: category(3), int64(4), object(4)
memory usage: 40.3+ KB


Так как есть столбец с учетом зарплаты в долларах США, то нет необходимости в столбцах salary и salary_currency.

In [198]:
data = data.drop(['salary', 'salary_currency'], axis=1)

In [199]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, 0 to 606
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           543 non-null    int64   
 1   experience_level    543 non-null    category
 2   employment_type     543 non-null    category
 3   job_title           543 non-null    object  
 4   salary_in_usd       543 non-null    int64   
 5   employee_residence  543 non-null    object  
 6   remote_ratio        543 non-null    int64   
 7   company_location    543 non-null    object  
 8   company_size        543 non-null    category
dtypes: category(3), int64(3), object(3)
memory usage: 31.8+ KB


По итогу отчистки данных имеем 543 строки и 9 столбцов данных без дубликатов и выбросов по зарплате.

## 2. Разведывательный анализ данных

### 2.1 Визуальный анализ данных

Для начала предлагаю посмотреть какие вообще вакансии есть в столбце job_title.

In [200]:
data['job_title'].unique()

array(['Data Scientist', 'Machine Learning Scientist',
       'Big Data Engineer', 'Product Data Analyst',
       'Machine Learning Engineer', 'Data Analyst', 'Lead Data Scientist',
       'Business Data Analyst', 'Lead Data Engineer', 'Lead Data Analyst',
       'Data Engineer', 'BI Data Analyst', 'Director of Data Science',
       'Research Scientist', 'Machine Learning Manager',
       'Data Engineering Manager', 'Data Science Consultant',
       'Machine Learning Infrastructure Engineer', 'ML Engineer',
       'AI Scientist', 'Computer Vision Engineer',
       'Principal Data Scientist', 'Data Science Manager', 'Head of Data',
       'Data Analytics Engineer', 'Applied Data Scientist',
       'Marketing Data Analyst', 'Cloud Data Engineer',
       'Financial Data Analyst', 'Computer Vision Software Engineer',
       'Director of Data Engineering', 'Data Science Engineer',
       'Principal Data Engineer', 'Machine Learning Developer',
       'Applied Machine Learning Scientist', 'D

Для начала необходимо определиться с критериями отнеcения тех или иных людей к позицииям Data Scientist или Data Engineer. Так как в исследовании нас интересуют только эти позиции.

Предлагаю считать специалистами Data Scientist всех, чья позиция содержит "Scie"

Предлагаю считать специалистами Data Engineer всех, чья позиция содержит "Engineer". Инженера будем искать первым, так как существует вероятноять появления вакансии 'Scie... Engineer'.

In [201]:
def job_separate(x):
    if 'Engineer' in x:
        x = 'Data Engineer'
    elif 'Scie' in x:
        x = 'Data Scientist'
    else:
        x = 'other'
    return x
     
    

In [202]:
data['job_title'] = data['job_title'].apply(job_separate)

In [203]:
data['job_title'].unique()

array(['Data Scientist', 'Data Engineer', 'other'], dtype=object)

In [204]:
data['job_title'] = data['job_title'].astype('category')

Дл общего понимания структуры выводим отчет

In [205]:
profile = ProfileReport(data, title="Pandas Profiling Report")

profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

***Выполним базовый анализ признаков.***

In [206]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='work_year',
    # labels={'salary_in_usd': "зарплата (USD)", 'y': 'Распределение'},
    title='Распределение количества данных по годам',
    )

fig.show()

Почти половина данных взята за 2022 год. 

In [207]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='experience_level',
    title='Распределение количества данных по уровню опыта',
    )

fig.show()

Среди рассмотренных позиций большинство представляют урвень Middle или Senior. Начинающих и управленцев среди выборки не так много

In [208]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='employment_type',
    title='Распределение количества данных по типу занятости',
    )

fig.show()

Подавляющее число позиций в позиций выборки это позиции на полный рабочий день. Количество позиций на фтриланс, контракт и неполный рабочий день исчезающе мало. рассматривать этот признак как информативный не имеет смысла. Столбец можно было бы удалить, но пока оставим.

In [209]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='job_title',
    title='Распределение количества данных по двум выбранным позицим',
    )

fig.show()

Число специалистов Data Engineer и Data Scientist примерно одинаково. 

In [210]:
fig = px.histogram(
    data[data['job_title'] != 'other'],
    x = 'salary_in_usd',
    labels={'salary_in_usd': "зарплата (USD)", 'y': 'Распределение'},
    title='Распределение зароботной платы в долларах США',
    marginal='box',
    )

fig.show()

Как уже говорилось ранее уровень зарплат имеет логнормальное распределение.

In [211]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='employee_residence',
    title='Распределение количества данных по странам проживания сотрудника',
    )

fig.show()

Как следует из диаграммы, половина сотрудников из выборки проживает в США. Чуть больше 30% проживает в Европе. остальные разбросаны по миру в пренебрежимо малых количествах. 

In [212]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='remote_ratio',
    title='Распределение количества данных по количеству работы, выполняемой удаленнно',
    )

fig.show()

58% сотрудников из выборки находятся на полно удаленке. И только 22.3% работают полный день в офисе. 

In [213]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='company_location',
    title='Распределение количества данных по локации нанимающих компаний',
    )

fig.show()

Здесь почти та же ситуация как с локацией сотрудников: Более половины в США, 30% в Европе.

In [214]:
fig = px.pie(
    data[data['job_title'] != 'other'],
    names='company_size',
    title='Распределение количества данных по размеру нанимающих компаний',
    )

fig.show()

47.9% процентов сотрудников работают в комипаниях среднего размера, 36.6 в крупных, 15.5 в маленьких.

**Выводы:** Исходя из того, что количество данных по годам увеличивается вочти в два раза с каждым годом, можно очень осторожно предположить, что растет количество сотрудников на позициях Data Scientist и Data Engineer. Предположение конечно требует проверки.

Несмотря на то, что половина рассмотренных сотрудников работает удаленно, они все равно проживают в странах компании нанимателя. По крайней мере в большинстве случаев. 

### 2.2 Визуальный анализ данных (вопросы исследования)

В данном разделе будут рассмотрены следующие вопросы:

- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?

In [215]:
salary_data = data[data['job_title'] == 'Data Scientist'].groupby('work_year', as_index=False)['salary_in_usd'].median()
salary_data

,work_year,salary_in_usd
0,2020,79833.0
1,2021,89327.0
2,2022,140200.0


In [216]:
fig = px.bar(
    salary_data,
    x = 'work_year',
    y = 'salary_in_usd',
    labels={'salary_in_usd': "зарплата (USD)", 'work_year': 'Год'},
    title='Медианная заработная плата специалистов Data Scientist по годам',
    )

fig.show()

**Выводы:** Медианная заработная плата специалиста Data Science за 2020 год составила 79833.0, за 2021 - 89327.0, за 2022 - 140200.0. Предварительно можно сделать вывод, что зарплата растет. Причем больше уровня инфляции . Если изменение заработной платы с 2020 по 2021 можно списать на инфляцию и вообще поставить под сомнение, то изменение с 2021 по 2022 (в 1,58 раза) на инфляцию уже списать не получится. Возможно в 2022 появился недостаток специалистов.

- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?

In [217]:
data_scientist_22 = data[(data['job_title'] == 'Data Scientist') & (data['work_year'] == 2022)]['salary_in_usd'].median()
data_engineer_22 = data[(data['job_title'] == 'Data Engineer') & (data['work_year'] == 2022)]['salary_in_usd'].median()
display(f'Заработная плата в 2022 года специалиста Data Scientist - {data_scientist_22}', f'Заработная плата в 2022 года специалиста Data Engineer - {data_engineer_22}')

'Заработная плата в 2022 года специалиста Data Scientist - 140200.0'

'Заработная плата в 2022 года специалиста Data Engineer - 117988.0'

In [218]:
fig = px.box(
    data[(data['job_title'] != 'other') & (data['work_year'] == 2022)],
    x = 'salary_in_usd',
    y = 'job_title',
    labels={'salary_in_usd': "зарплата (USD)", 'job_title': 'Позиция'},
    title='Распределение зароботных плат по позициям в 2022 году',
    color='job_title'
    )

fig.show()

**Выводы:** Медианная заработная плата специалиста Data Scientist за 2022 в 1.19 раза выше чем медианная заработная плата специалиста Data Engineer в том же году. Максимальна заработная плата Data Scientist в 1.56 раза больше чем у Data Engineer. Минимальная зараплата Data Engineer больше минимальной зарплаты Data Scientist в 1.08 раза.

Однако по распределению не так однозначно. В целом разброс заработных плат у Data Scientist больше чем у Data Engineer. Распределение для Data Engineer больше похоже на нормальное чем для Data Scientist. Вопрос требует дальнейшей проработки.

- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?

In [219]:
fig = px.box(
    data[data['job_title'] == 'Data Scientist'],
    x = 'salary_in_usd',
    y = 'company_size',
    labels={'salary_in_usd': "зарплата (USD)", 'company_size': 'Размер компании нанимателя'},
    title='Распределение зароботных плат по размеру компании нанимателя',
    color='company_size'
    )

fig.show()

**Выводы:** Наименьшую мединанную заработную. плату и и минимальное распределение зарплат имеют специалисты работающие в малых компаниях. Средние компании имеют наибольший разброс по зарплатам и наибольшую максимыльную заработную плату. Большие компании имеют немного меньший разброс по зарлатам для своих специалистов, зато имеется некоторое количество специалистов имеющих зарплату знаичительно выше среднего (скорее всего топ менеджеры).

Наименьшую минимальную зарплату имеют сотрудники работающие в малых компаниях. Минимальная заработная плата сотрудников средних компаний чуть выше. Максимальную стартовую заработную плату имеют сотрудники, работающие в крупных компаниях. Связываю это с тем, что чем больше компания тем сложнее туда попасть новичку. 

Мединанная заработная плата больше всего в средних компаниях. Немного меньше мединная зарплата в крупных компаниях и почти в два раза меньше медианная заработная плата в маленьких компаниях. Опять таки связываю это с трудность ттрудоустройства в средние и крупные компании.

- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

In [220]:
company_count = data[data['job_title'] != 'other'].groupby(['company_size', 'job_title'], as_index=False)['salary_in_usd'].count()
company_count

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\597617894.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,company_size,job_title,salary_in_usd
0,L,Data Engineer,66
1,L,Data Scientist,85
2,L,other,0
3,M,Data Engineer,112
4,M,Data Scientist,86
5,M,other,0
6,S,Data Engineer,34
7,S,Data Scientist,30
8,S,other,0


In [221]:
fig = px.bar(
    company_count,
    x = 'company_size',
    y = 'salary_in_usd',
    labels={'salary_in_usd': "Количество сотрудников в выборке", 
            'company_size': 'Размер компании нанимателя', 
            'job_title' : 'Позиция'},
    title='Количество сотрудников в выборке в зависимости от размера компании по позициям',
    color='job_title'
    )

fig.show()

**Выводы:** Исходя из полученных данных мы не можем сказать, есть ли компании (на важно какого размера), где в штате нет специалистов Data Scientist или Data Engineer. Однако предварително можем увидеть соотношение количества Data Scientist и Data Engineer в зависимости от размера компании. В маленьких компаниях соотношение Data Scientist и Data Engineer примерно 1:1, в средних  - 0.77, в крупных - 1.29. 
Можно предположить, что крупные компании реже занимаются обработкой данных сами. Средние, наоборот занимаются преимущественно обработкой данных. Мелкие же вынуждены делать все самостоятельно и держать в штате и тех и тех. 

### 2.3 Визуальный анализ данных (дополнительные исследования)

В данном разделе будут рассмотрены следующие вопросы:


- Есть ли зависимость заработной платы Data Scientist и Data Engineer от страны пребывания сотрудника?
- Есть ли зависимость заработной платы от страны нанимающей компании?
- Есть ли зависимость вывода сотрудника на удаленный режим работы от опытности сотрудника?

- Есть ли зависимость заработной платы Data Scientist и Data Engineer от страны пребывания сотрудника?

In [222]:
fig = px.box(
    data[data['job_title'] != 'other'],
    x = 'salary_in_usd',
    y = 'employee_residence',
    labels={'salary_in_usd': "Количество сотрудников в выборке", 
            'employee_residence': 'Страна проживания сотрудника', 
            'job_title' : 'Позиция'},
    title='Распределение зароботных плат в зависимости от страны проживания сотрудника',
    color='job_title'
    )

fig.show()

**Выводы:** Как и следовало ожидать, заработная плата сотрудников, проживающих в США значительно выше, чем у сотрудников в любых других странах. Почти в полтора раза меньше заработная плата в Великобритании и Германии. Причем почему-то в этих странах разброс зарплат Data Engineer значительно выше.

Удивительно, что остальные страны не идут с таким сильным отрывом по зарплате от Европейских стран, как Европа от США.


- Есть ли зависимость заработной платы от страны нанимающей компании?

In [223]:
fig = px.box(
    data[data['job_title'] != 'other'],
    x = 'salary_in_usd',
    y = 'company_location',
    labels={'salary_in_usd': "Количество сотрудников в выборке", 
            'company_location': 'Страна нанимающей компании', 
            'job_title' : 'Позиция'},
    title='Распределение зароботных плат в зависимости от страны нанимающей компании',
    color='job_title'
    )

fig.show()

**Выводы:** Распределение почти такое же, как и для стран проживания сотрудников. Можно предположить, что сотрудники в основном работают в стране проживания.

- Есть ли зависимость вывода сотрудника на удаленный режим работы от опытности сотрудника?

In [224]:
data['remote_ratio'].astype('category')
exp_count = data[data['job_title'] != 'other'].groupby(['experience_level', 'remote_ratio'], as_index=False)['salary_in_usd'].count()
exp_count

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\2019235399.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,experience_level,remote_ratio,salary_in_usd
0,EN,0,9
1,EN,50,19
2,EN,100,35
3,EX,0,3
4,EX,50,3
5,EX,100,13
6,MI,0,38
7,MI,50,34
8,MI,100,82
9,SE,0,42


In [225]:
fig = px.bar(
    exp_count,
    x = 'experience_level',
    y = 'salary_in_usd',
    labels={'experience_level': "Опыт сотрудника", 
            'remote_ratio': 'Режим удаленной работы', 
            'salary_in_usd' : 'Количество сотрудников'},
    title='Зависимость режима удаленной работы от опыта сотрудника',
    color='remote_ratio'
    )

fig.show()

**Выводы:** Наибольшее соотношение по количеству сотрудников между полной удаленкой и работой в офисе наблюдается у специалистов начинающего уроня (3.88). У сотрудников среднего (2.16) и Сениор уровня (2.67)  это соотношение падает.

### 2.4 Статистический анализ данных (вопросы исследования)

- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?

Предварительный вывод модно разделить на 2 утверждения:
1. Заработная плата 2020-2021 не изменялась
2. Заработная плата 2021-2022 увеличилась

Проверяем данные на нормальность

In [226]:
# задаём уровень значимости
alpha = 0.05 

# загружаем данные
salary_2020 = data[(data['work_year']==2020) & (data['job_title']== 'Data Scientist')]['salary_in_usd']
salary_2021 = data[(data['work_year']==2021) & (data['job_title']== 'Data Scientist')]['salary_in_usd']
salary_2022 = data[(data['work_year']==2022) & (data['job_title']== 'Data Scientist')]['salary_in_usd']

# проводим тест Шапиро — Уилка
_, p_2020 = stats.shapiro(salary_2020)
_, p_2021 = stats.shapiro(salary_2021)
_, p_2022 = stats.shapiro(salary_2022)

print(f'p-value 2020 = {p_2020:0.3f}, p-value 2021 = {p_2021:0.3f}, p-value 2022 = {p_2022:0.3f}')

# интерпретируем результат
if p_2020 <= alpha:
    print('Распределение 2020 не нормальное')
else:
    print('Распределение 2020 нормальное')
    
if p_2021 <= alpha:
    print('Распределение 2021 не нормальное')
else:
    print('Распределение 2021 нормальное')
    
if p_2022 <= alpha:
    print('Распределение 2022 не нормальное')
else:
    print('Распределение 2022 нормальное')



p-value 2020 = 0.000, p-value 2021 = 0.000, p-value 2022 = 0.025
Распределение 2020 не нормальное
Распределение 2021 не нормальное
Распределение 2022 не нормальное


Распределение не нормально. Для двух независимых групп применяем критерий Манна-Уитни.

1. Заработная плата 2020-2021 не изменялась

*Нулевая гипотеза: заработная плата в 2020 равна заработной плате в 2021*

*Альтернативная гипотеза: заработная плата в 2020 меньше заработной платы в 2021*

In [239]:
# проводим тест
_, p = stats.mannwhitneyu(salary_2020, salary_2021, alternative='less')

print(f'p-value = {p:0.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:0.3f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:0.3f}. У нас нет оснований отвергнуть нулевую гипотезу.')


p-value = 0.765
p-значение больше, чем заданный уровень значимости 0.050. У нас нет оснований отвергнуть нулевую гипотезу.


У нас нет основания отвергнуть нулевую гипотезу, значит, исходя из данных визуального анализа можем считать, что заработная плата с 2020 по 2021 не изменялась.

2. Заработная плата 2021-2022 увеличилась

*Нулевая гипотеза: заработная плата в 2021 больше или равна заработной плате в 2022*

*Альтернативная гипотеза:заработная плата в 2021 меньше заработной плате в 2022*

In [228]:
# проводим тест
_, p = stats.mannwhitneyu(salary_2021, salary_2022, alternative='less')

print(f'p-value = {p:0.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:0.3f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:0.3f}. У нас нет оснований отвергнуть нулевую гипотезу.')

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.050. Отвергаем нулевую гипотезу.


**Выводы:** Исходя из визуального анализа было выявлено, что заработная плата в 2020-2021 выросла незначительно, однако это оказалось это изменение не является статистически значимым. Так же подтвердилась гипотеза о том, что заработная плата в 2021-2022 годах росла (при визуальном анализе виден значительный рост). 

- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?

Из визуализации было видно, что в 2022 году заработная плата специалиста Data Scientist больше чем у специалиста Data Engineer.

Проверяем данные на нормальность

In [229]:
# загружаем данные
salary_ds = data[(data['work_year']==2022) & (data['job_title']== 'Data Scientist')]['salary_in_usd']
salary_de = data[(data['work_year']==2022) & (data['job_title']== 'Data Engineer')]['salary_in_usd']

# проводим тест Шапиро — Уилка
_, p_ds = stats.shapiro(salary_ds)
_, p_de = stats.shapiro(salary_de)


print(f'p-value Data Scientist = {p_ds:0.3f}, p-value Data Engineer = {p_de:0.3f}')

# интерпретируем результат
if p_ds <= alpha:
    print('Распределение Data Scientist не нормальное')
else:
    print('Распределение Data Scientist нормальное')
    
if p_de <= alpha:
    print('Распределение Data Engineer не нормальное')
else:
    print('Распределение Data Engineer нормальное')


p-value Data Scientist = 0.025, p-value Data Engineer = 0.010
Распределение Data Scientist не нормальное
Распределение Data Engineer не нормальное


Распределение не нормально. Для двух независимых групп применяем критерий Манна-Уитни.

*Нулевая гипотеза: заработная плата Data Scientist меньше или равна заработной плате Data Engineer*

*Альтернативная гипотеза: заработная плата Data Scientist больше заработной плате Data Engineer*

In [230]:
# проводим тест
_, p = stats.mannwhitneyu(salary_ds, salary_de, alternative='greater')

print(f'p-value = {p:0.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:0.3f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:0.3f}. У нас нет оснований отвергнуть нулевую гипотезу.')

p-value = 0.024
p-значение меньше, чем заданный уровень значимости 0.050. Отвергаем нулевую гипотезу.


**Выводы:** Исходя из визуального анализа было выявлено, что в 2022 году заработная плата Data Scientist была больше заработной платы Data Engineer. Статистический тест подтвердил наблюдения визуального анализа.

- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?

На основании визуального анализа было выявлено, неравенство заработных плат в компаниях разного размера.

Проверяем данные на нормальность

In [242]:
# загружаем данные
salary_s = data[(data['company_size']=='S') & (data['job_title']== 'Data Scientist')]['salary_in_usd']
salary_m = data[(data['company_size']=='M') & (data['job_title']== 'Data Scientist')]['salary_in_usd']
salary_l = data[(data['company_size']=='L') & (data['job_title']== 'Data Scientist')]['salary_in_usd']

# проводим тест Шапиро — Уилка
_, p_s = stats.shapiro(salary_s)
_, p_m = stats.shapiro(salary_m)
_, p_l = stats.shapiro(salary_l)


print(f'p-value Data Scientist = {p_ds:0.3f}, p-value Data Engineer = {p_de:0.3f}')

# интерпретируем результат
if p_s <= alpha:
    print('Распределение для компаний размера S не нормальное')
else:
    print('Распределение для компаний размера S нормальное')
    
if p_m <= alpha:
    print('Распределение для компаний размера M не нормальное')
else:
    print('Распределение для компаний размера M нормальное')
    
if p_l <= alpha:
    print('Распределение для компаний размера L не нормальное')
else:
    print('Распределение для компаний размера L нормальное')

p-value Data Scientist = 0.025, p-value Data Engineer = 0.063
Распределение для компаний размера S не нормальное
Распределение для компаний размера M не нормальное
Распределение для компаний размера L не нормальное


Распределение не нормально. Для трех независимых групп применяем критерий Краскела-Уоллиса.

*Нулевая гипотеза: заработная плата в компаниях разного размера одинакова.*

*Альтернативная гипотеза: заработная плата в компаниях разного размера неодинакова.*

In [232]:
# проводим тест
_, p = stats.kruskal(p_s, p_m, p_l)

print(f'p-value = {p:0.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:0.3f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:0.3f}. У нас нет оснований отвергнуть нулевую гипотезу.')

p-value = 0.368
p-значение больше, чем заданный уровень значимости 0.050. У нас нет оснований отвергнуть нулевую гипотезу.


**Выводы:** Исходя из визуального анализа было выявлено различие в заработных платах в компаниях разного размера. Статический тест показал, что разница не является статистически значимой. 

Заработная плата в компаниях разного размера не имеет статистически значимой разницы.

- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

Из визуального анализа было видно, что соотношение позиций Data Scientist и Data Engineer несколько отличается в зависимости от размера компании.

Для категориальных признаков используем критерий хи-квадрат.


*Нулевая гипотеза: Нет зависимости между размером компании и наличием должностей Data Scientist и Data Engineer.*

*Альтернативная гипотеза: Зависимость между наличием должностей Data Scientist и Data Engineer и размером компании есть.*

In [233]:
# вычисляем таблицу сопряжённости
table = pd.crosstab(data['job_title'], data['company_size'])

# проводим тест
_, p, _, _ = stats.chi2_contingency(table)

print(f'p-value = {p:.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:.2f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:.2f}. У нас нет оснований отвергнуть нулевую гипотезу.')


p-value = 0.002
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Выводы:** Статический тест подтвердил сделанное на основе визуального анализа предположение, что есть некоторые отклонения количества позиций Data Scientist и Data Engineer в зависимости от размера компании.

### 2.5 Статистический анализ данных (дополнительные исследования)

- Есть ли зависимость заработной платы Data Scientist и Data Engineer от страны пребывания сотрудника?

Исходя из визуального анализа данных мы предположили, что зарплаты Data Scientist и Data Engineer не одинаковы в различных странах. Расположили их так: США, Великобритания, Германия и другие.

Проверяем данные на нормальность

In [234]:
# загружаем данные
employe_data = data[(data['job_title']== 'Data Scientist')|(data['job_title']== 'Data Engineer')]

salary_us = employe_data[data['employee_residence']=='US']['salary_in_usd']
salary_gb = employe_data[data['employee_residence']=='GB']['salary_in_usd']
salary_de = employe_data[data['employee_residence']=='DE']['salary_in_usd']
salary_oth = employe_data[data['employee_residence']=='other']['salary_in_usd']

# проводим тест Шапиро — Уилка
_, p_us = stats.shapiro(salary_us)
_, p_gb = stats.shapiro(salary_gb)
_, p_de = stats.shapiro(salary_de)
_, p_oth = stats.shapiro(salary_oth)


print(f'p-value для США = {p_us:0.3f}\np-value для Великобритании = {p_gb:0.3f}\n\
p-value для Германии = {p_de:0.3f}\np-value для остальных стран = {p_oth:0.3f}\n\n')

# интерпретируем результат
if p_us <= alpha:
    print('Распределение для США не нормальное')
else:
    print('Распределение для США нормальное')
    
if p_gb <= alpha:
    print('Распределение для Великобритании не нормальное')
else:
    print('Распределение для Великобритании нормальное')
    
if p_de <= alpha:
    print('Распределение для Германии не нормальное')
else:
    print('Распределение для Германии нормальное')
    
if p_oth <= alpha:
    print('Распределение для остальных стран не нормальное')
else:
    print('Распределение для  остальных стран нормальное')

p-value для США = 0.000
p-value для Великобритании = 0.064
p-value для Германии = 0.126
p-value для остальных стран = 0.000


Распределение для США не нормальное
Распределение для Великобритании нормальное
Распределение для Германии нормальное
Распределение для остальных стран не нормальное


C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\2013232424.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\2013232424.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\2013232424.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\2013232424.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Распределение не нормально. Группы независимы. Для трех независимых групп применяем критерий Краскела-Уоллиса.

*Нулевая гипотеза: заработная плата не зависит от страны проживания сотрудника.*

*Альтернативная гипотеза: заработная плата различается в зависимости от страны проживания сотрудника.*

In [235]:
# проводим тест
_, p = stats.kruskal(salary_us, salary_gb, salary_de, salary_oth)

print(f'p-value = {p:0.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:0.3f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:0.3f}. У нас нет оснований отвергнуть нулевую гипотезу.')

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.050. Отвергаем нулевую гипотезу.


**Выводы:** Статический тест подтвердил сделанное на основе визуального анализа предположение, что имеется различие между зароботной платой в зависимоти от страны проживания сотрудника 

- Есть ли зависимость заработной платы от страны нанимающей компании?

Аналогично предыдущему тесту предположили, что имеется разница по зароботной плате в зависимости от страны нанимающей компании. Наибольшие зарпланы выплачиваются компаниями, расположенными в США. Далее по уменьшению зарплат следуют компании расположенные в Великобритании, Германии и других странах.

Проверяем данные на нормальность

In [236]:
# загружаем данные
employe_data = data[(data['job_title']== 'Data Scientist')|(data['job_title']== 'Data Engineer')]

salary_us = employe_data[data['company_location']=='US']['salary_in_usd']
salary_gb = employe_data[data['company_location']=='GB']['salary_in_usd']
salary_de = employe_data[data['company_location']=='DE']['salary_in_usd']
salary_oth = employe_data[data['company_location']=='other']['salary_in_usd']

# проводим тест Шапиро — Уилка
_, p_us = stats.shapiro(salary_us)
_, p_gb = stats.shapiro(salary_gb)
_, p_de = stats.shapiro(salary_de)
_, p_oth = stats.shapiro(salary_oth)


print(f'p-value для США = {p_us:0.3f}\np-value для Великобритании = {p_gb:0.3f}\n\
p-value для Германии = {p_de:0.3f}\np-value для остальных стран = {p_oth:0.3f}\n\n')

# интерпретируем результат
if p_us <= alpha:
    print('Распределение для США не нормальное')
else:
    print('Распределение для США нормальное')
    
if p_gb <= alpha:
    print('Распределение для Великобритании не нормальное')
else:
    print('Распределение для Великобритании нормальное')
    
if p_de <= alpha:
    print('Распределение для Германии не нормальное')
else:
    print('Распределение для Германии нормальное')
    
if p_oth <= alpha:
    print('Распределение для остальных стран не нормальное')
else:
    print('Распределение для  остальных стран нормальное')

p-value для США = 0.000
p-value для Великобритании = 0.053
p-value для Германии = 0.063
p-value для остальных стран = 0.000


Распределение для США не нормальное
Распределение для Великобритании нормальное
Распределение для Германии нормальное
Распределение для остальных стран не нормальное


C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\3186940751.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\3186940751.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\3186940751.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\zhigu\AppData\Local\Temp\ipykernel_18612\3186940751.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Распределение не нормально. Группы не зависимы. Выборки не зависимы. Для трех независимых групп применяем критерий Краскела-Уоллиса.

*Нулевая гипотеза: заработная плата не зависит от страны нанимающей компании.*

*Альтернативная гипотеза: заработная плата различается в зависимости от страны нанимающей компании.*

In [237]:
# проводим тест
_, p = stats.kruskal(salary_us, salary_gb, salary_de, salary_oth)

print(f'p-value = {p:0.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:0.3f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:0.3f}. У нас нет оснований отвергнуть нулевую гипотезу.')

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.050. Отвергаем нулевую гипотезу.


**Выводы:** Статический тест подтвердил сделанное на основе визуального анализа предположение, что имеется различие между зароботной платой в зависимоти от страны нанимающей компании.

- Есть ли зависимость вывода сотрудника на удаленный режим работы от опытности сотрудника?

При рассмотрении этого вопроса был сделан вывод, что есть зависимость между опытом и выводом сотрудника на удаленную работу.

Для категориальных признаков используем критерий хи-квадрат.


*Нулевая гипотеза: Нет зависимости между между опытом сотрудника и его переводом на удаленную работу.*

*Альтернативная гипотеза: Зависимость между опытом сотрудника и его переводом на удаленную работу есть.*

In [238]:
employe_data
# вычисляем таблицу сопряжённости
table = pd.crosstab(employe_data['experience_level'], data['remote_ratio'])

# проводим тест
_, p, _, _ = stats.chi2_contingency(table)

print(f'p-value = {p:.3f}')

# интерпретируем результат
if p <= alpha:
    print(f'p-значение меньше, чем заданный уровень значимости {alpha:.2f}. Отвергаем нулевую гипотезу.')
else:
    print(f'p-значение больше, чем заданный уровень значимости {alpha:.2f}. У нас нет оснований отвергнуть нулевую гипотезу.')


p-value = 0.045
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Выводы:** Как и показал визуальный анализ данных есть слабая зависимоть между опытом работы сотрудника и его переводом на удаленную работу.

## 3. Итоговые выводы

### 3.1 Итоговые выводы (вопросы исследования)

- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?

**Выводы:** Медианная заработная плата специалиста Data Science за 2020 год составила 79833.0, за 2021 - 89327.0, за 2022 - 140200.0. Предварительно был сделан вывод, что зарплата растет с 2020 по 2021. Однако статический тене подтвердил это утверждение. Статическим тестом подтвержден рост зарплат в период с 2021 по 2022 (в 1,58 раза согласно визуальному анализу). 

Можно предположить, что в 2022 году проявился недрстаток специалистов в профессии, что и привело к значительному росту заработной платы.


- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?

**Выводы:** Медианная заработная плата специалиста Data Scientist за 2022 в 1.19 раза выше чем медианная заработная плата специалиста Data Engineer в том же году. Максимальна заработная плата Data Scientist в 1.56 раза больше чем у Data Engineer. Минимальная зараплата Data Engineer больше минимальной зарплаты Data Scientist в 1.08 раза. Статический тест подтвердил утверждение, что заработная плата специалиста Data Scientist выше чем у специалиста Data Engineer.

Однако по распределению все не так однозначно. В целом разброс заработных плат у Data Scientist больше чем у Data Engineer. Распределение для Data Engineer больше похоже на нормальное, чем для Data Scientist. 

- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?

**Выводы:** Наименьшую мединанную заработную. плату и и минимальное распределение зарплат имеют специалисты работающие в малых компаниях. Средние компании имеют наибольший разброс по зарплатам и наибольшую максимыльную заработную плату. Большие компании имеют немного меньший разброс по зарлатам для своих специалистов, зато имеется некоторое количество специалистов имеющих заррплату знаичительно выше среднего (скорее всего топ менеджеры).

Наименьшую минимальную зарплату имеют сотрудники работающие в малых компаниях. Минимальная заработная плата сотрудников средних компаний чуть выше. Максимальную стартовую заработную плату имеюи сотрудниуи работающие в крупных компаниях. Связываю это с тем, что чем больше компания тем сложнее туду попасть новичку. 

Мединанная заработная плата больше всего в средних компаниях. Немного меньше мединная зарплата в крупных компаниях и почти в два раза меньше медианная заработная платита в маленьких компаниях. Опять таки связываю это с трудность ттрудоустройства в средние и крупные компании.

Исходя из визуального анализа было выявленоразличие в заработных платах в компаниях разного размера. Статический тест показал, что разница не является статистически значимой. 

Заработная плата в компаниях разного размера не имеет статистически значимой разницы.

- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

**Выводы:** Исходя из данных визуального анализа мы не можем сказать, есть ли компани (на важно какого размера), где в штате нет специалистов Data Scientist или Data Engineer. Однако предварително можем увидеть соотношение количества Data Scientist и Data Engineer в зависимости от размера компании. В маленьких компаниях соотношение Data Scientist и Data Engineer примерно 1:1, в средних  - 0.77, в крупных - 1.29. Статический тест подтвердил сделанное на основе визуального анализа предположение, что есть некоторые отклонения количества позиций Data Scientist и Data Engineer в зависимости от размера компании.

Можно предположить, что крупные компании реже занимаются обработкой данных сами. Средние, наоборот занимаются преимущественно обработкой данных. Мелкие же вынуждены делать все самостоятельно и держать в штате и тех и тех. 

### 3.2 Итоговые выводы (дополнительные исследования)

- Есть ли зависимость заработной платы Data Scientist и Data Engineer от страны пребывания сотрудника?
- Есть ли зависимость заработной платы от страны нанимающей компании?
- Есть ли зависимость вывода сотрудника на удаленный режим работы от опытности сотрудника?

- Есть ли зависимость заработной платы Data Scientist и Data Engineer от страны пребывания сотрудника?

**Выводы:** Как и следовало ожидать, заработная плата сотрудников, проживающих в США значительно выше чем у сотрудников в любых других странах. Почти в полтора раза меньше заработная плата в Великобритании и Германии. Причем почему-то в этих странах разброс зарплат Data Engineer значительно выше. Статический тест подтвердил сделанное на основе визуального анализа предположение, что имеется различие между зароботной платой в зависимоти от страны проживания сотрудника 

Удивительно, что остальные страны не идут с таким сильным отрывом по зарплате от Европейских странн, как Европа от США.

- Есть ли зависимость заработной платы от страны нанимающей компании?

**Выводы:** Распределение почти такое же, как и для стран проживания сотрудников. Можно предположить, что сотрудники в основном работают в стране проживания. Статический тест подтвердил сделанное на основе визуального анализа предположение, что имеется различие между зароботной платой в зависимоти от страны нанимающей компании.

- Есть ли зависимость вывода сотрудника на удаленный режим работы от опытности сотрудника?

**Выводы:** Наиболшее соотношение между полной удаленкой и работой в офисе наблюдается у специалистов начинающего уроня (3.88). У сотрудников среднего (2.16) и Сениор уровня (2.67)  это соотношение падает.

Как и показал визуальный анализ данных есть слабая зависимоть между опытом работы сотрудника и его переводом на удаленную работу.